In [0]:
# COMMAND ----------
# 1. CONFIGURATION SYNAPSE 
# ==============================================================================
synapse_server = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
synapse_user = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
synapse_pass = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"  
synapse_db = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"             

# Storage Info
storage_account = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
container_name = "curated"

# JDBC URL
jdbc_url = f"jdbc:sqlserver://{synapse_server}:1433;database={synapse_db};user={synapse_user};password={synapse_pass};encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;"

# Dossier Temporaire
temp_dir = f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/synapse_staging_temp"

# COMMAND ----------
# 2. AUTHENTICATION
# ==============================================================================
STORAGE_KEY = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

spark.conf.set(
    f"fs.azure.account.key.{storage_account}.dfs.core.windows.net",
    STORAGE_KEY
)

# COMMAND ----------
# 3. LECTURE DEPUIS GOLD
# ==============================================================================
gold_path = f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/gold_pollution_complete"

print(f"📂 Lecture depuis : {gold_path}")
df_gold = spark.read.format("delta").load(gold_path)

# COMMAND ----------
# 4. ECRITURE VERS SYNAPSE
# ==============================================================================
table_name = "dbo.AirQuality_Predictions"

print(f"🚀 Envoi vers Synapse Table : {table_name}...")

try:
    df_gold.write \
      .format("com.databricks.spark.sqldw") \
      .option("url", jdbc_url) \
      .option("forwardSparkAzureStorageCredentials", "true") \
      .option("dbTable", table_name) \
      .option("tempDir", temp_dir) \
      .mode("overwrite") \
      .save()
      
    print("✅ Data est dans la DB.")

except Exception as e:
    print(f"❌ Erreur Synapse : {e}")
    raise e